<a href="https://colab.research.google.com/github/PabloParadaSouto/Automatica/blob/master/aprendizaje_profundo_robotica_adaptativa_datos_robot_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

En primer lugar deberéis subir los archivos de datos (lecturas.npy y velocidades.npy) a vuetra cuenta de drive, colgarlos directamente del directorio raíz para que os funcione el código más abajo. Los dos primeros bloques de código solo ejecutarlos una única vez


In [ ]:
from google.colab import drive


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.regularizers import l2
import matplotlib.pyplot as plt
import numpy as np

drive.mount('/content/drive')
input ("montado, pulsa para seguir y cargar los datos ")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
montado, pulsa para seguir y cargar los datos 


''

Cargamos los datos, en principio esto sería suficiente con hacerlo una única vez....Se trata de datos grabados del Turtlebot (robot real) mientras seguía una pared, dado que la velocidad lineal es constante vamos a crear un modelo que aprenda a decidir la velocidad angular del robot en funcioń de lo que se percibe desde el láser.

In [ ]:
print ("voy a cargar datos sensores ")
sensores=np.load('/content/drive/MyDrive/lecturas.npy')
print ("cargado ...., ahora voy con velocidades")
velocidades=np.load('/content/drive/MyDrive/velocidades.npy')
print ("hecho ")

voy a cargar datos sensores 
cargado ...., ahora voy con velocidades
hecho 


Ahora normalizamos los datos, revisa como como lo hago, en principio esta celda solo se debería ejecutar una vez


In [ ]:
#En primer lugar hacemos una inspeccion visual de dimensiones para comprobar que todo correcto
print ("dimension de velocidades ",velocidades.shape," y dimension ",velocidades.ndim)
print ("dimension de lecturas ",sensores.shape," y dimension ",sensores.ndim)


sensores=sensores.astype('float32')
velocidades=velocidades.astype('float32')

#Invertimos los sensores para que tenga mas importancia lo que esta cerca
#hacemos luego la normalizacion habitual
sensores=np.reciprocal(sensores)
for i in range(sensores.shape[1]):
    sensores[:,i]=(sensores[:,i]-np.mean(sensores[:,i]))/np.std(sensores[:,i])
    print ("se ha normalizado ",i," el nuevo valor medio ",np.mean(sensores[:,i])," desviacion ",np.std(sensores[:,i]))


velocidades_maximas=np.amax(velocidades, axis=0)
velocidades_minimas=np.amin(velocidades, axis=0)
print ("velocidades_maximas ", velocidades_maximas)
print ("velocidades minimas ",velocidades_minimas)

maximav=np.amax(velocidades[:,1])
minimav=np.amin(velocidades[:,1])

velocidades[:,1]=(velocidades[:,1]-minimav)/(maximav-minimav)
print ("se ha normalizado  angular el nuevo valor medio ",np.mean(velocidades[:,1])," desviacion ",np.std(velocidades[:,1]))
print  ("Ya he finalizado toda la normalizacion ")


dimension de velocidades  (13427, 2)  y dimension  2
dimension de lecturas  (13427, 1438)  y dimension  2
se ha normalizado  0  el nuevo valor medio  -5.0002736e-08  desviacion  0.99999994
se ha normalizado  1  el nuevo valor medio  3.6365627e-08  desviacion  1.0000001
se ha normalizado  2  el nuevo valor medio  3.6365627e-08  desviacion  1.0000001
se ha normalizado  3  el nuevo valor medio  -4.5457034e-09  desviacion  1.0
se ha normalizado  4  el nuevo valor medio  -6.818555e-08  desviacion  0.99999994
se ha normalizado  5  el nuevo valor medio  4.5457034e-09  desviacion  0.99999994
se ha normalizado  6  el nuevo valor medio  -3.6365627e-08  desviacion  1.0
se ha normalizado  7  el nuevo valor medio  -5.909414e-08  desviacion  0.99999994
se ha normalizado  8  el nuevo valor medio  -1.8182813e-08  desviacion  0.99999994
se ha normalizado  9  el nuevo valor medio  9.091407e-09  desviacion  1.0
se ha normalizado  10  el nuevo valor medio  -1.8182813e-08  desviacion  0.99999994
se ha norm

Construimos los conjuntos de entrenamiento y test....

In [ ]:
cardinal_entrena=(int)(round(sensores.shape[0]*(3/4)))
print  ("numero de patrones de entrenamiento ",cardinal_entrena)
x_train=sensores[0:cardinal_entrena,:]
y_train=velocidades[0:cardinal_entrena,1]
y_train=y_train.reshape(y_train.shape[0],1)


x_test=sensores[cardinal_entrena:sensores.shape[0],:]
y_test=velocidades[cardinal_entrena:velocidades.shape[0],1]
y_test=y_test.reshape(y_test.shape[0],1)

print ("shape de entradas entrenamiento ",x_train.shape)
print ("shape de salidas entrenamiento ",y_train.shape)
print ("shape de entradas test ",x_test.shape)
print ("shape de salidas test ",y_test.shape)

numero de patrones de entrenamiento  10070
shape de entradas entrenamiento  (10070, 1438)
shape de salidas entrenamiento  (10070, 1)
shape de entradas test  (3357, 1438)
shape de salidas test  (3357, 1)


Ahora es el momento de crear una red neuronal, en este caso te toca a tí encontrar la topología de red más adecuada, yo solo pongo la estructura pero te toca fijar si quieres más capas, neuronas por capa, función de activación, etc...

In [ ]:
#definimos valores importantes
epocas=62
tamanho_batch=100

capa_oculta_1= 128 #ojo, este valor representa el numero de neuronas de la primera capa, tienes que cambiarlo, un cero no vale...

#verbose indica lo que se ve durante el entrenamiento
#By setting verbose 0, 1 or 2 you just say how do you want to 'see' the training progress for each epoch.
#verbose=0 will show you nothing (silent)
#verbose=1 will show you an animated progress bar
Verbose=2     #(one line per epoch)
opcion_muestra=1


neuronas_salida=1
probabilidad_dropout=0.3
division_validacion=0.2 #porcentaje de entrenamiento que se deja para validacion
dimension=sensores.shape[1] #esto representa el numero de columnas del patron sensorial

#Ahora construimos el modelo de red neuronal
modelo=tf.keras.models.Sequential()
modelo.add(keras.layers.Dense(capa_oculta_1,input_shape=(dimension,),name='capa_1',activation='tanh'))
modelo.add(keras.layers.Dropout(probabilidad_dropout))

modelo.add(keras.layers.Dense(75,name='capa_2',activation='tanh'))
modelo.add(keras.layers.Dropout(probabilidad_dropout))

modelo.add(keras.layers.Dense(20,name='capa_3',activation='tanh'))
modelo.add(keras.layers.Dropout(probabilidad_dropout))

#Fijate que en esta capa ponemos regularizacion L2, si no tienes claro lo que significa pregunta...
modelo.add(keras.layers.Dense(neuronas_salida,name='capa_4',activation='relu',kernel_regularizer=l2(0.01)))

#resumen del modelo
modelo.summary()
print ("ha llegado el momento de entrenar ... ")

# Es muy importante en que te fijes en el cambio de la funcion de loss, y metrics
#mse se usa sobre todo en regresion cuanso se quiere predecir un valor numerico
modelo.compile(optimizer='adam',loss='mean_squared_error', metrics=['mean_squared_error'])
#entrenamos el modelo
modelo.fit(x_train,y_train,batch_size=10,epochs=epocas,verbose=opcion_muestra,validation_split=division_validacion)
#evaluamos  el modelo
entrenamiento_loss,entrenamiento_acc=modelo.evaluate(x_train,y_train)
print ("resultado sobre entrenamiento  ",entrenamiento_acc, " loss ",entrenamiento_loss)
test_loss,test_acc=modelo.evaluate(x_test,y_test)
print ("resultado sobre test ",test_acc," loss ",test_loss)

input ("toma nota de los errores, ahora pusa para ver como funciona la red sobre el conjunto de test...")
salida_test=modelo(x_test)

for i in range(1000):
    print ("salida red ",(float)(salida_test[i])," salida deseada ",y_test[i])


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 capa_1 (Dense)              (None, 128)               184192    
                                                                 
 dropout_5 (Dropout)         (None, 128)               0         
                                                                 
 capa_2 (Dense)              (None, 75)                9675      
                                                                 
 dropout_6 (Dropout)         (None, 75)                0         
                                                                 
 capa_3 (Dense)              (None, 20)                1520      
                                                                 
 dropout_7 (Dropout)         (None, 20)                0         
                                                                 
 capa_4 (Dense)              (None, 1)                